In [1]:
pip install -U minsearch qdrant_client scikit-learn rouge tqdm pandas requests sentence-transformers


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.2/91.2 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 337.3/337.3 kB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 68.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 64.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.8/64.8 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 470.2/470.2 kB 31.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 74.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 48.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 37.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5

In [2]:
pip install jina

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.1/50.1 kB 4.9 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of types-requests to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of types-requests to determine which version is compatible with other requirements. This could take a while.
INFO: This is taking longer than usual. You might need to provide the dependency resolver with stricter constraints to reduce runtime. See https://pip.pypa.io/warnings/backtracking for guidance. If you want to abort this run, press Ctrl + C.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 71.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.8/302.8 kB 26.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.9/5.9 MB 114.2 MB/s eta 0:00:00

In [3]:
pip install qdrant-client

In [4]:
import requests
import pandas as pd

url_prefix = 'https://raw.githubusercontent.com/DataTalksClub/llm-zoomcamp/main/03-evaluation/'
docs_url = url_prefix + 'search_evaluation/documents-with-ids.json'
documents = requests.get(docs_url).json()

ground_truth_url = url_prefix + 'search_evaluation/ground-truth-data.csv'
df_ground_truth = pd.read_csv(ground_truth_url)
ground_truth = df_ground_truth.to_dict(orient='records')


In [5]:
from tqdm.auto import tqdm

def hit_rate(relevance_total):
    cnt = 0

    for line in relevance_total:
        if True in line:
            cnt = cnt + 1

    return cnt / len(relevance_total)

def mrr(relevance_total):
    total_score = 0.0

    for line in relevance_total:
        for rank in range(len(line)):
            if line[rank] == True:
                total_score = total_score + 1 / (rank + 1)

    return total_score / len(relevance_total)

def evaluate(ground_truth, search_function):
    relevance_total = []

    for q in tqdm(ground_truth):
        doc_id = q['document']
        results = search_function(q)
        relevance = [d['id'] == doc_id for d in results]
        relevance_total.append(relevance)

    return {
        'hit_rate': hit_rate(relevance_total),
        'mrr': mrr(relevance_total),
    }

In [6]:
import minsearch
from tqdm.auto import tqdm



index = minsearch.Index(
    text_fields=["question", "text", "section"],
    keyword_fields=["course", "id"]
)
index.fit(documents)

def q1_search(q):
    return index.search(
        query=q["question"],
        filter_dict={"course": q["course"]},
        boost_dict={"question": 1.5, "section": 0.1},
        num_results=5
    )

relevance = []
for q in tqdm(ground_truth):
    docs = q1_search(q)
    relevance.append([d["id"] == q["document"] for d in docs])

print("Q1 Hit Rate:", hit_rate(relevance))



  0%|          | 0/4627 [00:00<?, ?it/s]

Q1 Hit Rate: 0.848714069591528


In [7]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.pipeline import make_pipeline
from minsearch import VectorSearch

texts = [d["question"] for d in documents]
pipe = make_pipeline(TfidfVectorizer(min_df=3), TruncatedSVD(n_components=128, random_state=1))
Xq = pipe.fit_transform(texts)

vindex = VectorSearch(keyword_fields={"course"})
vindex.fit(Xq, documents)

def q2_search(q):
    vq = pipe.transform([q["question"]])
    return vindex.search(vq, filter_dict={"course": q["course"]}, num_results=5)

relevance = [[d["id"] == q["document"] for d in q2_search(q)] for q in tqdm(ground_truth)]
print("Q2 MRR:", mrr(relevance))

  0%|          | 0/4627 [00:00<?, ?it/s]

Q2 MRR: 0.3572833369353793


In [8]:
texts2 = [d["question"] + " " + d["text"] for d in documents]
Xqt = pipe.fit_transform(texts2)

vindex2 = VectorSearch(keyword_fields={"course"})
vindex2.fit(Xqt, documents)

def q3_search(q):
    vq = pipe.transform([q["question"]])
    return vindex2.search(vq, filter_dict={"course": q["course"]}, num_results=5)

relevance = [[d["id"] == q["document"] for d in q3_search(q)] for q in tqdm(ground_truth)]
print("Q3 Hit Rate:", hit_rate(relevance))

  0%|          | 0/4627 [00:00<?, ?it/s]

Q3 Hit Rate: 0.8210503566025502


In [15]:

# Qdrant client and models
from qdrant_client import QdrantClient
from qdrant_client.http.models import VectorParams, Distance, PointStruct, Filter, FieldCondition, MatchValue

# Embedding model
from sentence_transformers import SentenceTransformer

#  Initialize in-memory Qdrant
client = QdrantClient(":memory:")

# Load the Jina embedding model
model_handle = "jinaai/jina-embeddings-v2-small-en"
model = SentenceTransformer(model_handle, trust_remote_code=True)

#  Create a new collection with cosine distance
dim = model.get_sentence_embedding_dimension()
client.recreate_collection(
    collection_name="faq",
    vectors_config=VectorParams(size=dim, distance=Distance.COSINE)
)

# Prepare texts for indexing (question + text)
texts = [doc['question'] + ' ' + doc['text'] for doc in documents]

# Encode all at once (efficient batching)
embeddings = model.encode(texts, show_progress_bar=True)

# Upsert points into Qdrant
points = [
    PointStruct(
        id=int(doc["id"], 16),
        vector=emb.tolist(),
        payload={"doc_id": doc["id"], "course": doc["course"]}
    )
    for doc, emb in zip(documents, embeddings)
]
client.upsert(collection_name="faq", points=points)

def q4_search(q):
    # Use only the question as query (avoiding perfect self-match)
    qvec = model.encode([q['question']])[0]
    filt = Filter(must=[FieldCondition(key="course", match=MatchValue(value=q['course']))])
    hits = client.search(
        collection_name="faq",
        query_vector=qvec.tolist(),
        limit=5,
        query_filter=filt
    )
    # Extract hex IDs from payload
    return [{"id": hit.payload["doc_id"]} for hit in hits]

def hit_rate(rels):
    return sum(any(line) for line in rels) / len(rels)

def mrr(rels):
    total = 0
    for line in rels:
        for i, ok in enumerate(line):
            if ok:
                total += 1/(i+1)
                break
    return total / len(rels)

# Build relevance lists
relevance = [
    [d["id"] == q["document"] for d in q4_search(q)]
    for q in tqdm(ground_truth, desc="Evaluating Q4")
]

print("Q4 MRR:", round(mrr(relevance), 2))


/tmp/ipython-input-15-2232441724.py:17: DeprecationWarning: `recreate_collection` method is deprecated and will be removed in the future. Use `collection_exists` to check collection existence and `create_collection` instead.
  client.recreate_collection(


Batches:   0%|          | 0/30 [00:00<?, ?it/s]

Evaluating Q4:   0%|          | 0/4627 [00:00<?, ?it/s]

/tmp/ipython-input-15-2232441724.py:43: DeprecationWarning: `search` method is deprecated and will be removed in the future. Use `query_points` instead.
  hits = client.search(


Q4 MRR: 0.85


In [18]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.pipeline import make_pipeline

#  Load the evaluation results from gpt-4o-mini
url_prefix = 'https://raw.githubusercontent.com/DataTalksClub/llm-zoomcamp/main/03-evaluation/'
results_url = url_prefix + 'rag_evaluation/data/results-gpt4o-mini.csv'
df_results = pd.read_csv(results_url)

#  Create the TF-IDF + SVD embedding pipeline
pipeline = make_pipeline(
    TfidfVectorizer(min_df=3),
    TruncatedSVD(n_components=128, random_state=1)
)
pipeline.fit(df['answer_llm'] + ' ' + df['answer_orig'] + ' ' + df['question'])

# Define cosine similarity using  formula
def cosine(u, v):
    u_norm = np.sqrt(u.dot(u))
    v_norm = np.sqrt(v.dot(v))
    return u.dot(v) / (u_norm * v_norm)

#Compute cosine similarity for each answer pair
cos_similarities = []
for _, row in df.iterrows():
    v_llm = pipeline.transform([row['answer_llm']])[0]
    v_orig = pipeline.transform([row['answer_orig']])[0]
    cos_similarities.append(cosine(v_llm, v_orig))

# Calculate and display the average cosine similarity
avg_cos = sum(cos_similarities) / len(cos_similarities)
print("Q5 – Average A→Q→A Cosine Similarity:", round(avg_cos, 2))

Q5 – Average A→Q→A Cosine Similarity: 0.84


In [19]:
from rouge import Rouge

#  Initialize the ROUGE scorer
rouge_scorer = Rouge()

# Compute ROUGE-1 F1 for each answer pair
f1_scores = [
    rouge_scorer.get_scores(row['answer_llm'], row['answer_orig'])[0]['rouge-1']['f']
    for _, row in df_results.iterrows()
]

# Calculate the average ROUGE-1 F1 score
avg_f1 = sum(f1_scores) / len(f1_scores)
print("Q6 – Average ROUGE‑1 F1:", round(avg_f1, 2))

Q6 – Average ROUGE‑1 F1: 0.35
